# Ethereum

Using viem https://viem.sh/docs/introduction

## Keys

1. generation using [`@scure/bip39`](https://github.com/paulmillr/scure-bip39)
2. derivation using ...


In [ ]:
import * as bip39 from "npm:@scure/bip39"
import { wordlist } from "npm:@scure/bip39/wordlists/english.js"
const SEED_PHRASE = bip39.generateMnemonic(wordlist, 256)

In [ ]:
import { strict as assert } from "node:assert"
console.log(SEED_PHRASE)
assert(bip39.validateMnemonic(SEED_PHRASE, wordlist))

## Config

1. network https://ethereum.org/developers/docs/networks/
	* mainnet
	* sepolia
	* hoodi
	* ephemery

In [ ]:
import { http } from "viem"
const transport = http()
const chains = await import("viem/chains")
const NETWORK__ID = "ephemery" // "sepolia"
const chain = chains[NETWORK__ID] ?? {
	// https://ethereum.org/developers/docs/networks/#ephemery
	formatters: undefined,
	fees: undefined,
	serializers: undefined,
	id: 39438154, // From https://otter.bordel.wtf/
	name: "Ephemery",
	nativeCurrency: { name: "Ephemery Ether", symbol: "ETH", decimals: 18 },
	rpcUrls: { default: { http: [
		"https://otter.bordel.wtf/erigon"
		//"https://rpc.bordel.wtf/test"
	] } },
	blockExplorers: {
		default: {
			name: "Otterscan",
			url: "https://otter.bordel.wtf",
			apiUrl: "https://otter.bordel.wtf/api", // TODO check
		},
	},
	contracts: {
		multicall3: {
			address: "0xca11bde05977b3631167028862be2a173976ca11",
			blockCreated: 751532,
		},
		ensUniversalResolver: {
			address: "0xeeeeeeee14d718c2b47d9923deab1335e144eeee",
			blockCreated: 8928790,
		},
	},
	testnet: true,
};
assert(chain)
chain


## Addresses

Let's create a few to interact between themselves

In [ ]:
import { mnemonicToAccount } from "viem/accounts"

const account1 = mnemonicToAccount(SEED_PHRASE)

In [ ]:
const account2 = mnemonicToAccount(SEED_PHRASE, {
	accountIndex: 1,
})

## Clients

In [ ]:
import { createPublicClient, createWalletClient, http } from "viem"

const publicClient = createPublicClient({
	chain,
	transport,
});
console.log("publicClient =", publicClient);

const walletClient1 = createWalletClient({
	account: account1,
	chain,
	transport,
})
console.log("Client 1 =", walletClient1);

const walletClient2 = createWalletClient({
	account: account2,
	chain,
	transport,
});


## Status

In [ ]:
await(async (account, walletClient) => {
	const { address } = account;
	console.log("Address:");
	console.log(address);
	console.log(`${chain.blockExplorers.default.url}/address/${address}`);
	const balance = await publicClient.getBalance({ address });
	console.log("balance:", balance);
})(account1, walletClient1);


In [ ]:
await(async (account, walletClient) => {
	const { address } = account;
	console.log("Address:");
	console.log(address);
	console.log(`${chain.blockExplorers.default.url}/address/${address}`);
	const balance = await publicClient.getBalance({ address });
	console.log(balance);
})(account2, walletClient2);


## test

In [ ]:
const request = await client1.prepareTransactionRequest({
  account1,
  to: account2.address,
  value: 1000000000000000000n
})
console.log(request)

const serializedTransaction = await client1.signTransaction(request)
console.log(serializedTransaction);

const hash = await client1.sendRawTransaction({ serializedTransaction });
console.log(hash);